# 0 Begin chapter

## 0.1. install

In [12]:
!python -m pip install pandas pyarrow nltk

     ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
      --------------------------------------- 0.0/1.5 MB 320.0 kB/s eta 0:00:05
     - -------------------------------------- 0.1/1.5 MB 648.1 kB/s eta 0:00:03
     ------ --------------------------------- 0.3/1.5 MB 2.0 MB/s eta 0:00:01
     ------------------------ --------------- 0.9/1.5 MB 5.4 MB/s eta 0:00:01
     ---------------------------------------- 1.5/1.5 MB 7.4 MB/s eta 0:00:00
  Obtaining dependency information for regex>=2021.8.3 from https://files.pythonhosted.org/packages/b8/ad/3398312096118c4e62a5827664e52a04d5068e84d04142dd4a0da8a567ae/regex-2023.10.3-cp311-cp311-win_amd64.whl.metadata
     ---------------------------------------- 0.0/42.0 kB ? eta -:--:--
     ---------------------------------------- 42.0/42.0 kB ? eta 0:00:00
   ---------------------------------------- 0.0/269.6 kB ? eta -:--:--
   --------------------------------------- 269.6/269.6 kB 17.3 MB/s eta 0:00:00


## 0.2. Import

In [1]:
import pandas as pd
import numpy as np
import sys
import json
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.preprocessing import MinMaxScaler

from IPython.display import clear_output
from tqdm.notebook import tqdm
tqdm.pandas()
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 16 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


In [2]:
sys.version
# '3.11.6 (tags/v3.11.6:8b6ee5b, Oct  2 2023, 14:57:12) [MSC v.1935 64 bit (AMD64)]'

'3.11.6 (tags/v3.11.6:8b6ee5b, Oct  2 2023, 14:57:12) [MSC v.1935 64 bit (AMD64)]'

## 0.3. convert csv to parquet

In [11]:
hotels = pd.read_csv('data/hotels.zip')
# hotels.to_parquet('data/hotels_parquet.gzip', engine='pyarrow', compression='gzip')

## 0.4. Functions

In [4]:
def _get_col_names(df: pd.DataFrame, max_row_info: int=8) -> pd.DataFrame:
    ostatok = df.shape[1] % max_row_info
    col = df.shape[1] // max_row_info
    
    new_list = list()
    for index in range(col if not ostatok else col + 1):
        start = index*max_row_info
        end = (index + 1)*max_row_info
        new_list.append(df.columns[start:end].to_list())
    return pd.DataFrame((new_list))

In [5]:
# hotels = pd.read_parquet('data/hotels_parquet.gzip')
_get_col_names(hotels,3)

,0,1,2
0,hotel_address,additional_number_of_scoring,review_date
1,average_score,hotel_name,reviewer_nationality
2,negative_review,review_total_negative_word_counts,total_number_of_reviews
3,positive_review,review_total_positive_word_counts,total_number_of_reviews_reviewer_has_given
4,reviewer_score,tags,days_since_review
5,lat,lng,None


In [12]:
hotels = hotels.convert_dtypes()

In [13]:
hotels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 386803 entries, 0 to 386802
Data columns (total 17 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   hotel_address                               386803 non-null  string 
 1   additional_number_of_scoring                386803 non-null  Int64  
 2   review_date                                 386803 non-null  string 
 3   average_score                               386803 non-null  Float64
 4   hotel_name                                  386803 non-null  string 
 5   reviewer_nationality                        386803 non-null  string 
 6   negative_review                             386803 non-null  string 
 7   review_total_negative_word_counts           386803 non-null  Int64  
 8   total_number_of_reviews                     386803 non-null  Int64  
 9   positive_review                             386803 non-null  string 
 

---
- hotel_address — адрес отеля;
- review_date — дата, когда рецензент разместил соответствующий отзыв;
- average_score — средний балл отеля, рассчитанный на основе последнего комментария за последний год;
- hotel_name — название отеля;
- reviewer_nationality — страна рецензента;
- negative_review — отрицательный отзыв, который рецензент дал отелю;
- review_total_negative_word_counts — общее количество слов в отрицательном отзыв;
- positive_review — положительный отзыв, который рецензент дал отелю;
- review_total_positive_word_counts — общее количество слов в положительном отзыве.
- reviewer_score — оценка, которую рецензент поставил отелю на основе своего опыта;
- total_number_of_reviews_reviewer_has_given — количество отзывов, которые рецензенты дали в прошлом;
- total_number_of_reviews — общее количество действительных отзывов об отеле;
- tags — теги, которые рецензент дал отелю;
- days_since_review — количество дней между датой проверки и датой очистки;
- additional_number_of_scoring — есть также некоторые гости, которые просто поставили оценку сервису, но не оставили отзыв. Это число указывает, сколько там действительных оценок без проверки.
- lat — географическая широта отеля;
- lng — географическая долгота отеля.
---

# 1 Исследование данных

## 1.1. Отделение Категориальных признаков

In [14]:
resolution = hotels.shape[0]
tmp_cols = (
            hotels
            .nunique()
            .sort_values(ascending=False)
            .to_frame(name='count')
            .assign(frec=lambda x: round(100*x['count']/resolution).astype('UInt8'))
            )

cat_cols = (
            tmp_cols
            .query('frec < 20')
            .index
            .to_list()
            )

other_cols = (
                tmp_cols
                .query('frec >= 20')
                .index
                .to_list()
                )

print(f'cat_cols: {len(cat_cols)}; other_cols: {len(other_cols)}')


cat_cols: 15; other_cols: 2


## 1.2. Преобразование Категориальных признаков

### 1.2.1. Преобразование типов Object

In [15]:
object_cols = hotels[cat_cols].select_dtypes(include='string').columns.to_list()
# other_cat_cols = [x for x in cat_cols if x not in object_cols]
other_cat_cols = list(filter(lambda x: x not in object_cols, cat_cols))

hotels[object_cols].head()

,tags,hotel_address,hotel_name,days_since_review,review_date,reviewer_nationality
0,"[' Leisure trip ', ' Couple ', ' Studio Suite ...",Stratton Street Mayfair Westminster Borough Lo...,The May Fair Hotel,531 day,2/19/2016,United Kingdom
1,"[' Business trip ', ' Couple ', ' Standard Dou...",130 134 Southampton Row Camden London WC1B 5AF...,Mercure London Bloomsbury Hotel,203 day,1/12/2017,United Kingdom
2,"[' Leisure trip ', ' Solo traveler ', ' Modern...",151 bis Rue de Rennes 6th arr 75006 Paris France,Legend Saint Germain by Elegancia,289 day,10/18/2016,China
3,"[' Leisure trip ', ' Solo traveler ', ' Standa...",216 Avenue Jean Jaures 19th arr 75019 Paris Fr...,Mercure Paris 19 Philharmonie La Villette,681 day,9/22/2015,United Kingdom
4,"[' Business trip ', ' Couple ', ' Standard Dou...",Molenwerf 1 1014 AG Amsterdam Netherlands,Golden Tulip Amsterdam West,516 day,3/5/2016,Poland


In [16]:
# Преобразование столбца "tags"
hotels['tags'] = hotels['tags'].str.replace('\'','\"' ).apply(lambda x: json.loads(x))

# Преобразование признака "days_since_review"
hotels['days_since_review'] = hotels['days_since_review'].str.extract('(\d+) day', expand=False).astype('UInt16')

# Преобразование признака "review_date"
hotels['review_date'] = pd.to_datetime(hotels['review_date'], format='%d/%M/%Y').max()

# Преобразование признака  "reviewer_nationality"
hotels['hotel_name'] = hotels['hotel_name'].str.strip().replace('', np.nan)
category_order_list = sorted([x for x in hotels['hotel_name'].unique() if not pd.isnull(x)])
hotels['hotel_name'] = hotels['hotel_name'].astype('category').cat.set_categories(category_order_list, ordered=True)

# Преобразование признака  "reviewer_nationality"
hotels['reviewer_nationality'] = hotels['reviewer_nationality'].str.strip().replace('', np.nan)
category_order_list = sorted([x for x in hotels['reviewer_nationality'].unique() if not pd.isnull(x)])
hotels['reviewer_nationality'] = hotels['reviewer_nationality'].astype('category').cat.set_categories(category_order_list, ordered=True)

# Преобразование признака "hotel_address"
hotels['hotel_address'] = hotels['hotel_address'].astype('string')

### 1.2.2. Преобразование типов других типов

In [17]:
hotels[other_cat_cols].head()

,lat,lng,total_number_of_reviews,additional_number_of_scoring,review_total_negative_word_counts,review_total_positive_word_counts,total_number_of_reviews_reviewer_has_given,reviewer_score,average_score
0,51.507894,-0.143671,1994,581,3,4,7,10.0,8.4
1,51.521009,-0.123097,1361,299,3,2,14,6.3,8.3
2,48.845377,2.325643,406,32,6,0,14,7.5,8.9
3,48.888697,2.39454,607,34,0,11,8,10.0,7.5
4,52.385601,4.84706,7586,914,4,20,10,9.6,8.5


In [18]:
hotels['lat'] = hotels['lat'].astype('Float32')
hotels['lng'] = hotels['lng'].astype('Float32')
hotels['total_number_of_reviews'] = hotels['total_number_of_reviews'].astype('UInt16')
hotels['additional_number_of_scoring'] = hotels['additional_number_of_scoring'].astype('UInt16')
hotels['review_total_negative_word_counts'] = hotels['review_total_negative_word_counts'].astype('UInt16')
hotels['review_total_positive_word_counts'] = hotels['review_total_positive_word_counts'].astype('UInt16')
hotels['total_number_of_reviews_reviewer_has_given'] = hotels['total_number_of_reviews_reviewer_has_given'].astype('UInt16')
hotels['reviewer_score'] = hotels['reviewer_score'].astype('Float32')
hotels['average_score'] = hotels['average_score'].astype('Float32')

In [19]:
hotels[other_cat_cols].head()

,lat,lng,total_number_of_reviews,additional_number_of_scoring,review_total_negative_word_counts,review_total_positive_word_counts,total_number_of_reviews_reviewer_has_given,reviewer_score,average_score
0,51.507893,-0.143671,1994,581,3,4,7,10.0,8.4
1,51.521008,-0.123097,1361,299,3,2,14,6.3,8.3
2,48.845379,2.325643,406,32,6,0,14,7.5,8.9
3,48.888699,2.39454,607,34,0,11,8,10.0,7.5
4,52.385601,4.84706,7586,914,4,20,10,9.6,8.5


In [20]:
hotels['negative_review'] = hotels['negative_review'].astype('string')
hotels['positive_review'] = hotels['positive_review'].astype('string')

### 1.2.3. Работа с признаком "Tags"

In [ ]:
# a = (
#         hotels['tags']
#         .explode()
#         .str.strip()
#         .value_counts()
#         .to_frame()
#         )
# tags_unique_list = dict(zip(b:=(
#                                 pd.DataFrame(
#                                                 zip(
#                                                         a.index,
#                                                         scaler.fit_transform(a.values).flatten()
#                                                         ),
#                                                 columns=('tags', 'proportion')
#                                             )
#                                 .query('proportion > 0.2')['tags']
#                                 .values
#                                 ),
#                                 [False]*len(b)
#                             ))

In [21]:
tags_unique_list = dict(zip(a:=(
                            hotels['tags']
                            .explode()
                            .str.strip()
                            # .unique()
                            .value_counts()
                            .to_frame()
                            .assign(count=lambda x: x['count']/hotels.shape[0])
                            .query('count > 0.1')
                            .index
                            ),
                            [False]*len(a)
                            ))


 
def get_dummis_table(row: pd.Series, tags_unique_list: dict):
    import pandas as pd
    tmp_dict =tags_unique_list.copy()
    for item in list(map(lambda x: x.strip(), row)):
        if item in tmp_dict.keys():
            tmp_dict[item] = True
    return pd.Series(tmp_dict)


hotels = pd.concat((hotels, hotels['tags'].parallel_apply(get_dummis_table, args=(tags_unique_list,))), axis=1)
clear_output()
hotels.shape


(386803, 27)

### 1.2.4. Работа с отзывами.

In [22]:
nltk.download('stopwords')
nltk.download('punkt')

# Удаление пунктуации
def remove_punctuation(text: str):
    import string
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)

# Убираем стом слова
def remove_stopwords(text: str):
    from nltk.corpus import stopwords
    from nltk.tokenize import word_tokenize
    
    text = text.lower()
    stop_words = set(stopwords.words('english'))
    tokens = word_tokenize(text)
    filtered_text = [word for word in tokens if word not in stop_words]
    return ' '.join(filtered_text)

# Нормализуем окончания
def normalize_endings(text: str):
    from nltk.tokenize import word_tokenize
    from nltk.stem import SnowballStemmer
    
    stemmer = SnowballStemmer('english')
    tokens = word_tokenize(text)
    normalized_text = [stemmer.stem(word) for word in tokens]
    return ' '.join(normalized_text)


hotels['negative_review'] = hotels['negative_review'].progress_apply(remove_punctuation)
clear_output()

hotels['negative_review'] = hotels['negative_review'].parallel_apply(remove_stopwords)
clear_output()

hotels['negative_review'] = hotels['negative_review'].parallel_apply(normalize_endings)
clear_output()

hotels['positive_review'] = hotels['positive_review'].progress_apply(remove_punctuation)
clear_output()

hotels['positive_review'] = hotels['positive_review'].parallel_apply(remove_stopwords)
clear_output()

hotels['positive_review'] = hotels['positive_review'].parallel_apply(normalize_endings)
clear_output()

hotels['positive_review'] = hotels['positive_review'].str.split()
hotels['negative_review'] = hotels['negative_review'].str.split()

In [23]:
review_unique_dict = dict(zip(a:=(
                                pd.concat((hotels['positive_review'], hotels['negative_review']), axis=0, ignore_index=True)
                                .explode(ignore_index=True)
                                # .unique()
                                .value_counts()
                                .to_frame()
                                .assign(count=lambda x: x['count']/hotels.shape[0])
                                .query('count > 0.1')
                                .index
                                ),
                              [False]*len(a)
                              ))



def get_dummis_review_table(row: pd.Series, review_unique_dict: dict):
    import pandas as pd
    tmp_dict = review_unique_dict.copy()
    for item in row['positive_review']:
        if item in tmp_dict.keys():
            tmp_dict[item] = True
    for item in row['positive_review']:
        if item in tmp_dict.keys():
            tmp_dict[item] = True
    return pd.Series(tmp_dict)

hotels = pd.concat((hotels, hotels.parallel_apply(get_dummis_review_table, args=(review_unique_dict,), axis=1)), axis=1)
clear_output()
hotels.shape

(386803, 45)

In [24]:
hotels = (
            hotels
            .drop(columns=(
                            (hotels.iloc[:,17:].sum(axis=0)/hotels.shape[0])
                            .to_frame(name='proportion')
                            .query('proportion < 0.1')
                            .index
                        ))
            .drop(columns=['negative_review', 'positive_review', 'tags'])
        )

In [25]:
hotels.to_parquet('data/hotels_v2_parquet.gzip', engine='pyarrow', compression='gzip')

# 2. Подготовка данных для модели

In [28]:
hotels = pd.read_parquet('data/hotels_v2_parquet.gzip')

## Работа со строкой

In [18]:
# Удаление пунктуации
def remove_punctuation(text: str):
    import string
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)


hotels['negative_review'] = hotels['negative_review'].progress_apply(remove_punctuation)
clear_output()


import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('stopwords')
nltk.download('punkt')
def remove_stopwords(text: str):
    from nltk.corpus import stopwords
    from nltk.tokenize import word_tokenize
    stop_words = set(stopwords.words('english'))
    tokens = word_tokenize(text.lower())
    filtered_text = [word for word in tokens if word.lower() not in stop_words]
    return ' '.join(filtered_text)


hotels['negative_review'] = hotels['negative_review'].parallel_apply(remove_stopwords)
clear_output()


def normalize_endings(text: str):
    from nltk.tokenize import word_tokenize
    from nltk.stem import SnowballStemmer
    stemmer = SnowballStemmer('english')
    tokens = word_tokenize(text)
    normalized_text = [stemmer.stem(word) for word in tokens]
    return ' '.join(normalized_text)


hotels['negative_review'] = hotels['negative_review'].parallel_apply(normalize_endings)
clear_output()


# 2. Подготовка данных для модели

In [198]:
# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели  
# Х - данные с информацией об отелях, у - целевая переменная (рейтинги отелей)  
X = hotels.drop(['reviewer_score'], axis = 1)  
y = hotels['reviewer_score'] 

In [11]:
# Загружаем специальный инструмент для разбивки:  
from sklearn.model_selection import train_test_split  

In [12]:
# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.  
# Для тестирования мы будем использовать 25% от исходного датасета.  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [13]:
# Импортируем необходимые библиотеки:  
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели  
from sklearn import metrics # инструменты для оценки точности модели  
  
# Создаём модель  
regr = RandomForestRegressor(n_estimators=100)  
      
# Обучаем модель на тестовом наборе данных  
regr.fit(X_train, y_train)  
      
# Используем обученную модель для предсказания рейтинга отелей в тестовой выборке.  
# Предсказанные значения записываем в переменную y_pred  
y_pred = regr.predict(X_test)  


ValueError: could not convert string to float: 'Pla a de Llevant s n Sant Mart 08019 Barcelona Spain'

In [9]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они отличаются  
# Метрика называется Mean Absolute Percentage Error (MAPE) и показывает среднюю абсолютную процентную ошибку предсказанных значений от фактических.  
print('MAPE:', metrics.mean_absolute_percentage_error(y_test, y_pred))

MAPE: 0.14138286324223787


Небольшой бонус:


In [10]:
# # убираем признаки которые еще не успели обработать, 
# # модель на признаках с dtypes "object" обучаться не будет, просто выберим их и удалим
# object_columns = [s for s in hotels.columns if hotels[s].dtypes == 'object']
# hotels.drop(object_columns, axis = 1, inplace=True)

# # заполняем пропуски самым простым способом
# hotels = hotels.fillna(0)